In [1]:
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from tqdm import tnrange
import torch.optim as optim
import torch.nn.functional as F
from PROB_LVAE_shGLM import PROB_LVAE_shGLM
from sklearn import metrics
import torch.distributions as dist

# Hyperparams

In [2]:
train_T = 20000
test_T = 8000

C_den = torch.zeros(5,5)
C_den[0,1:] = 1
sub_no = C_den.shape[0]

batch_size = 1500

syn_basis_no = 2
hist_basis_no = 2
T_hist = 201
T_syn = 201
T_enc = 201
hid_dim = 256
temp = 0.5

In [3]:
Ensyn = torch.tensor([0, 106, 213, 211, 99])
Insyn = torch.tensor([1, 22, 36, 42, 19])
E_no = torch.sum(Ensyn)
I_no = torch.sum(Insyn)

C_syn_e = torch.zeros(sub_no, E_no)
C_syn_i = torch.zeros(sub_no, I_no)

E_count = 0
for s in range(sub_no):
    C_syn_e[s,E_count:E_count+Ensyn[s]] = 1
    E_count += Ensyn[s]

I_count = 0
for s in range(sub_no):
    C_syn_i[s,I_count:I_count+Insyn[s]] = 1
    I_count += Insyn[s]

# Train Params

In [4]:
model = PROB_LVAE_shGLM(C_den.cuda(), C_syn_e.cuda(), C_syn_i.cuda(), E_no, T_syn, syn_basis_no,
                T_hist, hist_basis_no, T_enc, hid_dim, temp)
model = model.float().cuda()

V_ref = np.fromfile("/media/hdd01/sklee/cont_shglm/inputs/vdata_NMDA_ApN0.5_13_Adend_r0_o2_i2_g_b0.bin")
V_ref = V_ref[1:-2]

train_V_ref = V_ref[:train_T]
test_V_ref = V_ref[train_T:train_T+test_T]

test_V_ref = torch.from_numpy(test_V_ref).cuda()
train_V_ref = torch.from_numpy(train_V_ref)

In [5]:
E_neural = np.load("/media/hdd01/sklee/cont_shglm/inputs/Espikes_d48000_r1_rep1_Ne629_e5_E20_neural.npy")
I_neural = np.load("/media/hdd01/sklee/cont_shglm/inputs/Ispikes_d48000_r1_rep1_Ni120_i20_I30_neural.npy")

train_S_E = E_neural[:train_T]
train_S_I = I_neural[:train_T]
test_S_E = E_neural[train_T:train_T+test_T]
test_S_I = I_neural[train_T:train_T+test_T]

train_S_E = torch.from_numpy(train_S_E)
train_S_I = torch.from_numpy(train_S_I)
test_S_E = torch.from_numpy(test_S_E).float().cuda()
test_S_I = torch.from_numpy(test_S_I).float().cuda()

In [6]:
repeat_no = 2
batch_no = (train_V_ref.shape[0] - batch_size) * repeat_no
train_idx = np.empty((repeat_no, train_V_ref.shape[0] - batch_size))
for i in range(repeat_no):
    part_idx = np.arange(train_V_ref.shape[0] - batch_size)
    np.random.shuffle(part_idx)
    train_idx[i] = part_idx
train_idx = train_idx.flatten()
train_idx = torch.from_numpy(train_idx)

print(batch_no)
print(train_idx.shape[0])

37000
37000


In [7]:
optimizer = optim.Adam(model.parameters(), lr=0.005)

In [ ]:
loss_array = np.empty((batch_no))
beta = 0

for i in tnrange(batch_no):
    
    if i%250 == 249 and i <= 2500:
        beta += 0.1
    
    model.train()
    optimizer.zero_grad()
    batch_idx = train_idx[i].long()
    batch_S_E = train_S_E[batch_idx : batch_idx+batch_size].float().cuda()
    batch_S_I = train_S_I[batch_idx : batch_idx+batch_size].float().cuda()
    batch_ref = train_V_ref[batch_idx:batch_idx+batch_size].float().cuda()
    
    batch_pred, rec_loss, KL_loss, Z_P_prior, Z_P_post = model(batch_ref,
                                                              batch_S_E,
                                                              batch_S_I,
                                                              beta)
        
    loss = rec_loss + beta* KL_loss
    loss_array[i] = loss.item()
    
    print(i, rec_loss.item(), KL_loss.item())
    
    loss.backward()
    optimizer.step()
    
    if i%50 == 0:
        model.eval()
        test_pred, test_Y, test_spikes, test_probs = model.decode(test_S_E, test_S_I)
        test_diff = (test_V_ref - test_pred) ** 1
        test_loss = torch.var(test_diff)
        test_score = metrics.explained_variance_score(y_true=test_V_ref.cpu().detach().numpy(),
                                                      y_pred=test_pred.cpu().detach().numpy(),
                                                      multioutput='uniform_average')
        train_score = metrics.explained_variance_score(y_true=batch_ref.cpu().detach().numpy(),
                                                      y_pred=batch_pred.cpu().detach().numpy(),
                                                      multioutput='uniform_average')
        print("TEST", i, test_loss.item(), test_score.item(), train_score.item())
        print(torch.mean(test_spikes, 0).cpu().detach().numpy())
        #print(torch.mean(spike_probs, 0).cpu().detach().numpy())
        if i%100 == 0:
            torch.save(model.state_dict(), "/media/hdd01/sklee/lvae_shglm/PROB_LVAE_sub5_s2_h2_shglm_i"+str(i)+".pt")
    


<ipython-input-8-9c2adff36d86>:4: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  for i in tnrange(batch_no):


0 8.576885223388672 0.3278140127658844
TEST 0 13.626896736016645 0.013352384812941764 -0.002111196517944336
[0.45698917 0.43162653 0.43744186 0.45981818]
1 1.7542930841445923 1.3282411098480225
2 7.713833808898926 1.3375247716903687
3 8.910249710083008 1.3137508630752563
4 4.583776950836182 1.2735886573791504
5 4.830690383911133 0.720924973487854
6 1.7461963891983032 0.42527419328689575
7 5.9067301750183105 0.7260293960571289
8 3.260881185531616 0.7169657349586487
9 13.589946746826172 0.7499551773071289
10 2.375617265701294 0.7407633662223816
11 8.980557441711426 0.7289440035820007
12 1.8983169794082642 0.7429654598236084
13 4.822018623352051 0.742756187915802
14 3.6344034671783447 0.7459501028060913
15 3.326178789138794 0.7231020331382751
16 2.5441668033599854 0.7590619921684265
17 21.171472549438477 0.7786175012588501
18 12.673247337341309 0.7743884921073914
19 4.559658527374268 0.7668285965919495
20 12.876668930053711 0.7692634463310242
21 7.095689296722412 0.776707112789154
22 1.61

190 11.123440742492676 0.8676260709762573
191 3.5478157997131348 0.7992351651191711
192 5.756288051605225 0.805017352104187
193 1.1406824588775635 0.7555655837059021
194 4.704470157623291 0.8405296206474304
195 5.572667121887207 0.8466939330101013
196 2.307523488998413 0.7566445469856262
197 6.79141092300415 0.8270826935768127
198 1.3169732093811035 0.8746285438537598
199 1.9506863355636597 0.7337161302566528
200 3.4109952449798584 0.728693425655365
TEST 200 10.929234529419478 0.20867506431529015 0.17303013801574707
[0.3863067  0.3702294  0.34316793 0.4601708 ]
201 1.0672320127487183 0.8528189063072205
202 9.741012573242188 0.820728600025177
203 1.9808595180511475 0.8641529679298401
204 2.2738263607025146 0.8080952167510986
205 19.275705337524414 0.8746135830879211
206 11.48500919342041 0.8323898911476135
207 14.227950096130371 0.8287854790687561
208 1.2666497230529785 0.8852530121803284
209 6.4744791984558105 0.8013144731521606
210 3.1219184398651123 0.758720874786377
211 3.9270582199

377 4.2797393798828125 0.47976651787757874
378 2.8480875492095947 0.36706066131591797
379 5.419889450073242 0.4482470154762268
380 2.7094502449035645 0.3744366765022278
381 3.801097869873047 0.4664090573787689
382 8.579838752746582 0.48482444882392883
383 3.899620294570923 0.4499942660331726
384 4.363779067993164 0.4771898686885834
385 1.3121947050094604 0.42038989067077637
386 5.137823104858398 0.4796864092350006
387 0.9620341658592224 0.47547677159309387
388 10.040367126464844 0.4727182686328888
389 0.8322218656539917 0.3904407024383545
390 6.273530006408691 0.46074026823043823
391 3.7835288047790527 0.4633849263191223
392 0.7955931425094604 0.4807577133178711
393 1.748371958732605 0.39956167340278625
394 18.050891876220703 0.475487619638443
395 13.032965660095215 0.45076581835746765
396 10.043985366821289 0.44917920231819153
397 1.516674518585205 0.3933907747268677
398 4.205774784088135 0.4246686100959778
399 8.600749015808105 0.4546900987625122
400 2.2915070056915283 0.381441086530

562 1.343268632888794 0.3489169478416443
563 11.200549125671387 0.3769727647304535
564 4.410172462463379 0.40500912070274353
565 1.9294365644454956 0.3002539575099945
566 4.216687202453613 0.3551924228668213
567 7.267853260040283 0.40117642283439636
568 2.856844663619995 0.35267505049705505
569 3.0856261253356934 0.3439667820930481
570 2.400479793548584 0.3535226881504059
571 1.2886747121810913 0.3056650459766388
572 2.1878292560577393 0.3477630913257599
573 4.525142192840576 0.3950272500514984
574 10.927094459533691 0.3667164742946625
575 3.4091849327087402 0.36636897921562195
576 1.0827281475067139 0.3236260414123535
577 1.4414385557174683 0.3107530176639557
578 8.973153114318848 0.3666127920150757
579 6.723700523376465 0.37029263377189636
580 2.362182378768921 0.2833611071109772
581 7.811596393585205 0.3850634694099426
582 7.218774318695068 0.3638610541820526
583 2.0700058937072754 0.3051445484161377
584 1.7302337884902954 0.3016338646411896
585 1.7049415111541748 0.3057948052883148

...

2023 1.3549299240112305 0.17576895654201508
2024 1.707668662071228 0.1923667937517166
2025 1.342896819114685 0.20827026665210724
2026 1.4135351181030273 0.17851927876472473
2027 4.950311660766602 0.22896787524223328
2028 2.242199659347534 0.22964602708816528
2029 1.1786346435546875 0.1823689490556717
2030 1.956435203552246 0.2372492104768753
2031 1.804547905921936 0.20964916050434113
2032 4.919524669647217 0.23456430435180664
2033 1.7844940423965454 0.20580647885799408
2034 1.1935820579528809 0.1921960860490799
2035 1.4678010940551758 0.18941649794578552
2036 1.1704024076461792 0.18973156809806824
2037 1.0001474618911743 0.189658522605896
2038 0.9184761047363281 0.1862931102514267
2039 1.1154142618179321 0.19342297315597534
2040 1.4597758054733276 0.2162620574235916
2041 1.093919038772583 0.1825701892375946
2042 4.671518802642822 0.2429075837135315
2043 1.0213546752929688 0.1881837099790573
2044 1.7710111141204834 0.19480672478675842
2045 0.8913134336471558 0.17587538063526154
2046 1.3

2202 1.1024086475372314 0.1766241043806076
2203 1.560534954071045 0.17186754941940308
2204 1.0768381357192993 0.17871545255184174
2205 17.875038146972656 0.25131097435951233
2206 1.0828723907470703 0.1783474087715149
2207 0.8684712052345276 0.17522835731506348
2208 0.8602427840232849 0.17213959991931915
2209 1.2965911626815796 0.17944122850894928
2210 1.6054341793060303 0.21162833273410797
2211 1.9510806798934937 0.17630207538604736
2212 4.014876842498779 0.21080701053142548
2213 1.8401964902877808 0.2170766144990921
2214 5.831693649291992 0.2093660533428192
2215 10.308335304260254 0.2277688980102539
2216 1.4341927766799927 0.1700182557106018
2217 1.013723611831665 0.1674264669418335
2218 1.2324466705322266 0.17287148535251617
2219 0.7861728072166443 0.15949110686779022
2220 5.775771617889404 0.20403540134429932
2221 1.0815858840942383 0.17192500829696655
2222 1.5476399660110474 0.1702863723039627
2223 1.425572395324707 0.17179998755455017
2224 1.953126311302185 0.194172665476799
2225 

2384 2.007582187652588 0.1775716096162796
2385 2.8235373497009277 0.16577669978141785
2386 4.217163562774658 0.18983536958694458
2387 1.5517934560775757 0.19633963704109192
2388 3.4706614017486572 0.18500719964504242
2389 0.9415816068649292 0.15839475393295288
2390 1.4322823286056519 0.17643822729587555
2391 3.8781723976135254 0.20573340356349945
2392 3.901740074157715 0.1931556612253189
2393 1.4741864204406738 0.15185050666332245
2394 3.4594693183898926 0.20732706785202026
2395 1.2883191108703613 0.15460467338562012
2396 0.6583163142204285 0.14840960502624512
2397 1.0162086486816406 0.15985001623630524
2398 1.2679939270019531 0.15288636088371277
2399 1.1393160820007324 0.1567259281873703
2400 0.8604260087013245 0.16089242696762085
TEST 2400 5.86880623152746 0.5750724628775189 0.527949333190918
[0.6254414  0.59412163 0.60220885 0.00173504]
2401 0.8396874070167542 0.15715394914150238
2402 1.72062349319458 0.18160469830036163
2403 1.042004942893982 0.16384615004062653
2404 1.028036475181

2563 1.0433597564697266 0.14749743044376373
2564 1.2000092267990112 0.16821475327014923
2565 1.6394026279449463 0.18110845983028412
2566 1.00140380859375 0.14533933997154236
2567 2.135408401489258 0.16759228706359863
2568 1.0305812358856201 0.14767533540725708
2569 1.1156680583953857 0.14818114042282104
2570 6.531844139099121 0.19288167357444763
2571 6.196218490600586 0.18862932920455933
2572 0.9004044532775879 0.15058813989162445
2573 0.9398457407951355 0.14352427423000336
2574 1.5607420206069946 0.14897538721561432
2575 2.412677049636841 0.1757650524377823
2576 4.030041694641113 0.19519199430942535
2577 1.2362499237060547 0.14179885387420654
2578 3.9066362380981445 0.19274117052555084
2579 3.6118125915527344 0.17568092048168182
2580 1.0792790651321411 0.13718053698539734
2581 0.9264976382255554 0.1443505883216858
2582 2.092440128326416 0.17158135771751404
2583 1.0873181819915771 0.14814524352550507
2584 5.65748929977417 0.17990510165691376
2585 2.470245361328125 0.17766672372817993
2

2745 1.05080246925354 0.13626880943775177
2746 0.7639002799987793 0.1363607794046402
2747 1.0613362789154053 0.14600232243537903
2748 0.9277602434158325 0.1303238570690155
2749 4.514950752258301 0.18322864174842834
2750 1.3103221654891968 0.1465671807527542
TEST 2750 5.015359722040465 0.6368657662393116 0.7194349765777588
[0.6759901  0.65004444 0.658579   0.00080088]
2751 1.9890104532241821 0.16307248175144196
2752 1.24668288230896 0.1494111269712448
2753 5.123989582061768 0.16871756315231323
2754 1.7228158712387085 0.1647731065750122
2755 2.4396893978118896 0.16237600147724152
2756 2.2772653102874756 0.12858468294143677
2757 1.3188726902008057 0.12880033254623413
2758 3.389202833175659 0.15756215155124664
2759 2.1203362941741943 0.1455407738685608
2760 1.000187873840332 0.13761533796787262
2761 1.496649980545044 0.16179266571998596
2762 0.787839412689209 0.13710585236549377
2763 1.0532361268997192 0.14287112653255463
2764 0.838712215423584 0.13247992098331451
2765 1.386994481086731 0.

In [ ]:
plt.plot(test_pred.cpu().detach().numpy())

In [ ]:
plt.plot(test_pred.cpu().detach().numpy())